## Outline

1. Create your environment with Google Cloud
2. Serve a Deep Learning model as an API using Keras, Flask, and Docker
3. Deploy said model with Kubernetes
4. Bask in the glory of your newfound knowledge


# 1. Create an Environment with Google Cloud

Create a virtual Machine on GCP

The next step is to select the compute size we want to use. <br> The default (read: cheapest) should work just fine

![alt text](https://cdn-images-1.medium.com/max/800/1*nUAYU6Ol3kc-BJF5gZ4_RA.png)

To start a Google Cloud VM, open up the ribbon on the left side of your screen. <br> 
Theb Select ``Compute Engine``.<br>
Then choose “Create Instance”. <br>
You can see in the photo below that I have one instance already running.

![image](https://cdn-images-1.medium.com/max/800/1*1cx80c0CgohQRvIYmkck9w.png)

The next step is to select the compute size we want to use. The default (read: cheapest) settings should work just fine.


![image](https://cdn-images-1.medium.com/max/800/1*TlybZKutOgD2xF4_vDt8rA.png)

Next I choose the operating system and disk space I want to use. Select “Boot Disk” to edit the defaults. I choose Centos 7 for my operating system and increase the amount of disk from 10GB to 100GB. My choice of operating system (Centos) is not required. I would recommend increasing the disk size beyond 10GBs, however, as the Docker containers we create are ~1GB each.

The final step before we create the VM is to set our Firewall rules to allow HTTP/S. Full transparency, I’m not sure if this step is required. I will show you how to edit the firewall settings to test our API on the VM before we deploy it to Kubernetes. So checking these boxes is not sufficient — there is more work to be done. I just haven’t gone back to try this tutorial again without checking them.

![image](https://cdn-images-1.medium.com/max/800/1*97xCXpVyFMnrk6k1m5ympQ.png)

Now click “Create”. Bravo! The hard part is basically done!

![image](https://cdn-images-1.medium.com/max/800/1*b80mKfRZXXbvWf77gwZX3Q.png)

# Step - 2 Install Docker into your VM

Now, let’s SSH into our VM and install Docker. 
<br>The easiest way to do this is to simply click the SSH icon next to your VM (below).
<br>This opens a terminal in your browser.

![image](https://cdn-images-1.medium.com/max/800/1*qRWFVHOiL5HJHWsXfSPMeQ.png)

You should be able to see a terminal window open in a new tab as below:

![image](https://github.com/BenjaminAkera/Talks/raw/master/res/init_terminal.png)


### 1. Remove the existing version of Docker

<pre>

sudo yum remove docker docker-client docker-client-latest docker-common docker-latest docker-latest-logrotate docker-logrotate docker-selinux docker-engine-selinux docker-engine

</pre>

### 2. Install latest version of Docker

<pre>
sudo yum install -y yum-utils device-mapper-persistent-data lvm2
sudo yum-config-manager -- add-repo https://download.docker.com/linux/centos/docker-ce.repo
sudo yum install docker-ce
</pre>

### 3. Start Docker and Run Test Script

<pre>
sudo systemctl start docker
sudo docker run hello-world
</pre>

<b> Expected Output:</b>

![image](https://github.com/BenjaminAkera/Talks/raw/master/res/hello_docker.png)

<br>

# Step 3 - Create your Deep Learning Model

This step consists of 3 parts:
Creating a machine learning model can be done in two ways: 
    1. You can build a model from scratch and train it eg: Regression/classification
    2. Use a pretrained model for transfer learning.
    
In this case we shall use a pre-trained model for inference.<br>
We shall use the <b>Resnet</b> Image classification model pretrained on ImageNet

### Convolutional Neural Network Model
![image](https://github.com/BenjaminAkera/Talks/raw/master/res/convnet.png)

# Step 4 - Serve the Resnet Pre-trained Model with Flask 

In this step we create a flask web application that serves the pretrianed tensorflow model.<br>
The model has already been trained on ImageNet and can classify 1000 classes of everyday objects<br>
We use the Keras wrapper over tensorflow to load the model file and pass it to helper functions to sanitize the inputs

Create the directory where your project will reside:``mkdir keras-app``<br>
change directory into the new directory: ``cd keras-app`` <br>
create a new file with nano ```nano app.py```<br>

Copy the code in the cell below and save it as ``app.py``

```python

# USAGE
# Start the server:
# 	python app.py
# Submit a request via cURL:
# 	curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'

# import the necessary packages
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io
import tensorflow as tf

# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None

def load_model():
	# load the pre-trained Keras model (here we are using a model
	# pre-trained on ImageNet and provided by Keras, but you can
	# substitute in your own networks just as easily)
	global model
	model = ResNet50(weights="imagenet")
	global graph
	graph = tf.get_default_graph()

def prepare_image(image, target):
	# if the image mode is not RGB, convert it
	if image.mode != "RGB":
		image = image.convert("RGB")

	# resize the input image and preprocess it
	image = image.resize(target)
	image = img_to_array(image)
	image = np.expand_dims(image, axis=0)
	image = imagenet_utils.preprocess_input(image)

	# return the processed image
	return image

@app.route("/predict", methods=["POST"])
def predict():
	# initialize the data dictionary that will be returned from the
	# view
	data = {"success": False}

	# ensure an image was properly uploaded to our endpoint
	if flask.request.method == "POST":
		if flask.request.files.get("image"):
			# read the image in PIL format
			image = flask.request.files["image"].read()
			image = Image.open(io.BytesIO(image))

			# preprocess the image and prepare it for classification
			image = prepare_image(image, target=(224, 224))

			# classify the input image and then initialize the list
			# of predictions to return to the client
			with graph.as_default():
				preds = model.predict(image)
				results = imagenet_utils.decode_predictions(preds)
				data["predictions"] = []

				# loop over the results and add them to the list of
				# returned predictions
				for (imagenetID, label, prob) in results[0]:
					r = {"label": label, "probability": float(prob)}
					data["predictions"].append(r)

				# indicate that the request was a success
				data["success"] = True

	# return the data dictionary as a JSON response
	return flask.jsonify(data)

# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
	print(("* Loading Keras model and Flask starting server..."
		"please wait until server has fully started"))
	load_model()
	app.run(host='0.0.0.0')
       


```

# Step 5 -  Dockerize the Flask Application

We Create a ```requirements.txt``` file that we can use to define the dependencies used by the flask application

### i. Create a Requirements.txt File

in your terminal type: ``` nano requirements.txt``` and paste the contents below

```text
keras
tensorflow
flask
gevent
pillow
requests
```

Save the text file in nano by pressing ``ctrl+O`` then ``ctrl+z`` to exit nano

### ii. Create a Docker File

Type in terminal: ``` touch Dockerfile ``` 
open the newly created ``Dockerfile`` with nano ie. ```nano Dockerfile```<br>
copy and paste the contents below and save the file

```
FROM python:3.6
WORKDIR /app
COPY requirements.txt /app
RUN pip install -r ./requirements.txt
COPY app.py /app
CMD ["python", "app.py"]~
```

Line1: instructing docker to download the base image of Python3 <br>
Line2: use the Python package manager pip to install the packages detailed in ```requirements.txt``` <br>
Line3: we then tell Docker to run our script via ```python app.py```


### iii. Build the Docker Image

```sudo docker build -t keras-app:latest .```

This instructs Docker to build a container for the code located in our current working directory keras-app


### iv. Run the Docker Container

```sudo docker run -d -p 5000:5000 keras-app```

A quick note about the numbers, 5000:5000 — here we are telling Docker to make port 5000 externally available and to forward our local app to that port (which is also running on port 5000 locally)


Check status of our container by running: ```sudo docker ps -a```


<b>Output:</b>
    
```
CONTAINER ID          IMAGE               COMMAND             CREATED             STATUS                PORTS              NAMES
d82f65802166        keras-app           "python app.py"     About an hour ago   Up About an hour          0.0.0.0:5000->5000/tcp   nervous_northcutt
```

### v. Test our model locally

With our model running, now it is time to test it. <br>
This model accepts as input a photo of a dog and returns the dog’s breed. 

Download the image of a dog (or any animal) and save it in your working directory. <br> You can use wget ie: ```Wget https://github.com/jrosebr1/simple-keras-rest-api/raw/master/dog.jpg```


![image](https://cdn-images-1.medium.com/max/800/1*FJWlc5VIb2k5Y64DgMZuww.jpeg)

From Terminal Run:
    
```curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'```


Make sure that “dog.jpg” is in your current directory (or provide the appropriate path to the file).

You should see a result  Like:


```json
{"predictions":[{"label":"beagle","probability":0.987775444984436},{"label":"pot","probability":0.0020967808086425066},{"label":"Cardigan","probability":0.001351703773252666},{"label":"Walker_hound","probability":0.0012711131712421775},{"label":"Brittany_spaniel","probability":0.0010085132671520114}],"success":true}
```

# Step 6 - Deploy Model to Kubernettes

<b>1. Create a docker hub account (if you dont have one)</b>

The first thing we do is upload our model to Docker Hub. (If you don’t have a Docker Hub account, create one now — don’t worry, it’s free).<br> The reason we do this is that we won’t physically move our container to our Kubernetes cluster.<br>
Instead, we will instruct Kubernetes to install our container from a centrally hosted server, i.e., **Docker Hub**.



<b>2. Login to your Docker Hub account</b>

Once you have created your Docker Hub account, log in from the command line via ```sudo docker login```. <br>
You’ll need to supply your username and password just as if you were logging into the website.


If you see a message like this:

 
```Login Succeeded```


Then you were able to login successfully. Now let’s move to the next step.

<b>3. Tag your container</b>

We need to tag our container before we can upload it. <br>
Think of this step as giving our container a name.<br>
First, run ```sudo docker images``` and locate the image id for our keras-app container.<br>
The output should look something like this:


 ```bash
 REPOSITORY   TAG     IMAGE ID     CREATED       SIZE 
 keras-app    latest  d1ae332f45cb About an hour ago   1.61GB
```

Now let’s tag our keras-app. Be sure to follow my formatting and replace the values for image id and docker hub id with your specific values.



```bash
#Format
sudo docker tag <your image id> <your docker hub id>/<app name>
#My Exact Command - Make Sure To Use Your Inputs
sudo docker tag d1ae332f45cb akeraben/keras-app
```

<b>4. Push our container to Docker Hub</b>

Now we can push our container. From the shell run:


```bash 
#Format
sudo docker push <your docker hub name>/<app-name>
#My exact command
sudo docker push akeraben/keras-app
```

Now if you navigate back to Docker Hub’s website, you should see your ``keras-app`` repository. 
<br>Well done! We’re ready for the final stretch.

<b>5. Create a Kubernetes Cluster</b>


From the Google Cloud home screen, select **Kubernetes Engine**


Next we’ll customize the size of the nodes in our cluster. <br>
I’ll select **4vCPUs with 15 GBs of RAM**. You can try this with a smaller cluster.<br>
Remember, the default settings spin up **3 nodes**, so your cluster will have **3X** the resources of what your provision, i.e., in my case 45 GBs of Ram.<br>
I’m being a bit lazy and choosing a larger size as we won’t have our kubernetes cluster running for very long.


After that, just click **Create**. It will take a minute or two for you cluster to spin up.<br>
Now let’s connect to the cluster. Click **Run** in Cloud Shell to bring up the console for the Kubernetes cluster.

Note that this is a separate shell environment from your VM where you created and tested your Docker container. We could install Kubernetes on VMs, but Google’s Kubernetes service automates that for us.
Now we run our docker container in Kubernetes. <br>
Note that the image tag is just pointing to our hosted docker image on Docker Hub.

In addition, we’ll specify with --port that we want to run our app on **port 5000**


```kubectl run keras-app --image=gcav66/keras-app --port 5000```

In Kubernetes, containers all run inside of pods. <br>
We can verify that our pod is running by typing kubectl get pods.
If you see something like this below, you’re all set.

```bash 
akeraben@cloudshell:~ (basic-web-app-test)$ kubectl get pods
NAME                         READY     STATUS    RESTARTS   AGE
keras-app-79568b5f57-5qxqk   1/1       Running   0          1m
```

Now that our pod is alive and running, we need to expose our pod on port 80 to the outside world. <br>This means that anyone visiting the IP address of our deployment can access our API. <br>It also means we don’t have to specify a pesky port number after our url like we did before (say goodbye to :5000).


```kubectl expose deployment keras-app --type=LoadBalancer --port 80 --target-port 5000```


We’re almost there! Now, we determine the status of our deployment (and the URL that we need to call our API) by running kubectl get service . Again, if the output of that command looks like what I have below, you’re doing great.


```
akeraben@cloudshell:~ (basic-web-app-test)$ kubectl get service
NAME         TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
keras-app    LoadBalancer   10.11.250.71   35.225.226.94   80:30271/TCP   4m
kubernetes   ClusterIP      10.11.240.1    <none>          443/TCP        18m
```

## Testing the model

Grab that cluster-ip for your keras application because now is the moment of truth. Open your local terminal (or wherever you have dog photo handy) and run the following command to call the API 


 
```curl -X POST -F image=@dog.jpg 'http://<your service IP>/predict'```


Feast your eyes on the results!
As you can see below, the API correctly returns the label of beagle for the picture.



```$ curl -X POST -F image=@dog.jpg 'http://35.225.226.94/predict'```

**output**

```json
{"predictions":[{"label":"beagle","probability":0.987775444984436},{"label":"pot","probability":0.0020967808086425066},{"label":"Cardigan","probability":0.001351703773252666},{"label":"Walker_hound","probability":0.0012711131712421775},{"label":"Brittany_spaniel","probability":0.0010085132671520114}],"success":true}
```



# Conclusion

Now, there are many improvements we could make to this project. For starters, we should change the python web server running our flask app from our local python server to something production-grade like **gunicorn/uWSGI**. <br>We should also explore the scaling and management features of Kubernetes, which we barely touched on. Finally, we might try building a kubernetes environment from scratch.


## Credit

Inspiration came from the Following awesome resources
<ls>

<li> 1.<a href="https://medium.com/analytics-vidhya/deploy-your-first-deep-learning-model-on-kubernetes-with-python-keras-flask-and-docker-575dc07d9e76">This Blog</a></li>
<li> 1.<a href="https://www.youtube.com/watch?v=1lgsQ3PKz9M">This Video By Siraj Raval</a></li>